In [1]:
import requests
from bs4 import BeautifulSoup
import json
import csv
import logging
import re
import time
import datetime
from datetime import date
from dateutil.parser import parse

In [2]:
# Setup the csv writer
nbafile = open("/kaggle/working//NBA_Season_avgs.csv", "w", newline='', encoding='utf-8')
nbawriter = csv.writer(nbafile)
# set headings
nbawriter.writerow(['RK', 
                    'Team', 
                    'Year', 
                    'G', 
                    'W', 
                    'L', 
                    'Conf', 
                    'MP', 
                    'FG', 
                    'FGA', 
                    'FG%', 
                    '3P',
                    '3PA', 
                    '3P%', 
                    '2P', 
                    '2PA', 
                    '2P%', 
                    'FT', 
                    'FTA', 
                    'FT%', 
                    'ORB', 
                    'DRB', 
                    'AST', 
                    'STL', 
                    'BLK', 
                    'TOV', 
                    'PF', 
                    'PTS'])

103

In [3]:
# Define Year for URL
start_year = 1980
end_year = 2023
start_year

1980

In [4]:
import requests

class NBAscrapy:
    
    runcount = 1
    rk = 1
    year = 1980
    
    # __init__ required to initialize the scrape tool
    def __init__(self):
        self.main()

    # starting URL according to start year above
    def main(self):
        print("Code is running!")
        url = "https://www.basketball-reference.com/leagues/NBA_"+str(start_year)+".html"
        self.getyearteamdata(url)
    def getyearteamdata(self, url):
        
        # running through each table by rank since website uses rank in each table used in ascending order
        # rank will reset to 1 once we move to a new year
        self.rk = 1
        print("--------------year: " + str(self.year) + " |" + " row: " + str(self.runcount) + "-----------------")
        
        # get data, total_pages
        page = requests.get(url)
        page.encoding = "UTF-8"
        pagecontent = str(page.content)
        pagecontent = pagecontent.replace('<!--',"")
        pagecontent = pagecontent.replace("-->", "")
        pagecontent = pagecontent.replace("\n", "")

        # print(pagecontent)
        soup = BeautifulSoup(pagecontent, 'html.parser')
        soup.encode('utf-8')
         # print(pagecontent)
        soup = BeautifulSoup(pagecontent, 'html.parser')
        soup.encode('utf-8')

        # format material for page.json()
        
        # data will be pulled from 'Team Per Game Stats' table (avg game data)
        Team = ""
        Year = self.year
        G = ""
        MP = ""
        FG = ""
        FGA = ""
        FGpercent = ""
        ThreeP = ""
        ThreePA = ""
        ThreePpercent = ""
        TwoP = ""
        TwoPA = ""
        TwoPpercent = ""
        FT = ""
        FTA = ""
        FTpercent = ""
        ORB = ""
        DRB = ""
        TRB = ""
        AST = ""
        STL = ""
        BLK = ""
        TOV = ""
        PF = ""
        PTS = ""
        
        # data will be pulled from 'Conference Standings' table
        W = ""
        L = ""
        Conf = ""
        E_W_teams = []
        
        
        # for testing we wanted to split between East and West
        # This required pulling from the 'Conference Standings' table
        # If statement to Eastern Conference
        if soup.find('table', {'id':'divs_standings_E'}) is not None:
            if soup.find('table', {'id':'divs_standings_E'}).findAll('tr', {'class': 'full_table'}) is not None:
                E_team_trs = soup.find('table', {'id':'divs_standings_E'}).findAll('tr', {'class': 'full_table'})
                nI = 0
                print("Code is running!")
                # Loop through Eastern Conference, collect wins, lossess
                for E_team_tr in E_team_trs:
                    E_team = {
                        'index': 'East', # Index the data by 'East' Conference
                        'teamname': E_team_tr.find('a').text.strip(),
                        'W': E_team_tr.find('td',{'data-stat': 'wins'}).text.strip(),
                        'L': E_team_tr.find('td',{'data-stat': 'losses'}).text.strip()
                    }
                    E_W_teams.append(E_team.copy())
                    nI += 1
                    
        # If statement to Western Conference
        if soup.find('table', {'id':'divs_standings_W'}) is not None:
            if soup.find('table', {'id':'divs_standings_W'}).findAll('tr', {'class': 'full_table'}) is not None:
                W_team_trs = soup.find('table', {'id':'divs_standings_W'}).findAll('tr', {'class': 'full_table'})
                nI = 0
                
                # Loop through Western Conference, collect wins, lossess
                for W_team_tr in W_team_trs:
                    W_team = {
                        'index': 'West', # Index the data by 'West' Conference
                        'teamname': W_team_tr.find('a').text.strip(),
                        'W': W_team_tr.find('td',{'data-stat': 'wins'}).text.strip(),
                        'L': W_team_tr.find('td',{'data-stat': 'losses'}).text.strip()
                    }
                    E_W_teams.append(W_team.copy())
                    nI += 1
                    
        # If statement to collect data from league "Team Per Game Stats" table            
        if soup.find('table', {'id':'team-stats-per_game'}) is not None:
            if soup.find('table', {'id':'team-stats-per_game'}).findAll('tr') is not None:
                Team_stats_trs = soup.find('table', {'id':'team-stats-per_game'}).findAll('tr')
                for Team_stats_tr in Team_stats_trs:
                    thhead = Team_stats_tr.find('th',{'scope': 'row'})
                    sheet = []
                    if thhead is not None:
                        Team = Team_stats_tr.find('td',{'data-stat': 'team_name'}).text.strip()
                        if Team != "League Average":
                            G = Team_stats_tr.find('td',{'data-stat': 'g'}).text.strip()
                            MP = Team_stats_tr.find('td',{'data-stat': 'mp'}).text.strip()
                            FG = Team_stats_tr.find('td',{'data-stat': 'fg'}).text.strip()
                            FGA = Team_stats_tr.find('td',{'data-stat': 'fga'}).text.strip()
                            FGpercent = Team_stats_tr.find('td',{'data-stat': 'fg_pct'}).text.strip()
                            ThreeP = Team_stats_tr.find('td',{'data-stat': 'fg3'}).text.strip()
                            ThreePA = Team_stats_tr.find('td',{'data-stat': 'fg3a'}).text.strip()
                            ThreePpercent = Team_stats_tr.find('td',{'data-stat': 'fg3_pct'}).text.strip()
                            TwoP = Team_stats_tr.find('td',{'data-stat': 'fg2'}).text.strip()
                            TwoPA = Team_stats_tr.find('td',{'data-stat': 'fg2a'}).text.strip()
                            TwoPpercent = Team_stats_tr.find('td',{'data-stat': 'fg2_pct'}).text.strip()
                            FT = Team_stats_tr.find('td',{'data-stat': 'ft'}).text.strip()
                            FTA = Team_stats_tr.find('td',{'data-stat': 'fta'}).text.strip()
                            FTpercent = Team_stats_tr.find('td',{'data-stat': 'ft_pct'}).text.strip()
                            ORB = Team_stats_tr.find('td',{'data-stat': 'orb'}).text.strip()
                            DRB = Team_stats_tr.find('td',{'data-stat': 'drb'}).text.strip()
                            TRB = Team_stats_tr.find('td',{'data-stat': 'trb'}).text.strip()
                            AST = Team_stats_tr.find('td',{'data-stat': 'ast'}).text.strip()
                            STL = Team_stats_tr.find('td',{'data-stat': 'stl'}).text.strip()
                            BLK = Team_stats_tr.find('td',{'data-stat': 'blk'}).text.strip()
                            TOV = Team_stats_tr.find('td',{'data-stat': 'tov'}).text.strip()
                            PF = Team_stats_tr.find('td',{'data-stat': 'pf'}).text.strip()
                            PTS = Team_stats_tr.find('td',{'data-stat': 'pts'}).text.strip()

                            # Confirm the Win, Loss, and Conference
                            e_w_confirm = self.check_E_W(E_W_teams, Team)
                            if e_w_confirm != "error":
                                W = e_w_confirm['W']
                                L = e_w_confirm['L']
                                Conf = e_w_confirm['index']
                                
                            # append ALL team data
                            sheet.append(self.rk)
                            sheet.append(Team)
                            sheet.append(Year)
                            sheet.append(G)

                            sheet.append(W)
                            sheet.append(L)
                            sheet.append(Conf)

                            sheet.append(MP)
                            sheet.append(FG)
                            sheet.append(FGA)
                            sheet.append(FGpercent)
                            sheet.append(ThreeP)
                            sheet.append(ThreePA)
                            sheet.append(ThreePpercent)
                            sheet.append(TwoP)
                            sheet.append(TwoPA)
                            sheet.append(TwoPpercent)
                            sheet.append(FT)
                            sheet.append(FTA)
                            sheet.append(FTpercent)
                            sheet.append(ORB)
                            sheet.append(DRB)
                            sheet.append(AST)
                            sheet.append(STL)
                            sheet.append(BLK)
                            sheet.append(TOV)
                            sheet.append(PF)
                            sheet.append(PTS)
                            self.rk += 1
                            print(sheet)
                            nbawriter.writerow(sheet)
                            print("--------------------------------------------------------")
                            print(" ")
                            

        print("--------------------------------------------------------")
        print(" ")
        print(E_W_teams)

        # move to next page to scrape if under year boundries
        if self.year <= (end_year-1):
            self.runcount += 1
            self.year += 1
            url = "https://www.basketball-reference.com/leagues/NBA_" + str(self.year) + ".html"
            self.getyearteamdata(url)

    def check_E_W(self, E_W_teams, teamname):
        t_name = "error"
        for E_W_team in E_W_teams:
            if E_W_team['teamname'] == teamname.replace("*",""):
                t_name = E_W_team
                break
        return t_name
    

NBAscrapy()




Code is running!
--------------year: 1980 | row: 1-----------------
Code is running!
--------------------------------------------------------
 
[{'index': 'East', 'teamname': 'Boston Celtics', 'W': '61', 'L': '21'}, {'index': 'East', 'teamname': 'Philadelphia 76ers', 'W': '59', 'L': '23'}, {'index': 'East', 'teamname': 'Washington Bullets', 'W': '39', 'L': '43'}, {'index': 'East', 'teamname': 'New York Knicks', 'W': '39', 'L': '43'}, {'index': 'East', 'teamname': 'New Jersey Nets', 'W': '34', 'L': '48'}, {'index': 'East', 'teamname': 'Atlanta Hawks', 'W': '50', 'L': '32'}, {'index': 'East', 'teamname': 'Houston Rockets', 'W': '41', 'L': '41'}, {'index': 'East', 'teamname': 'San Antonio Spurs', 'W': '41', 'L': '41'}, {'index': 'East', 'teamname': 'Indiana Pacers', 'W': '37', 'L': '45'}, {'index': 'East', 'teamname': 'Cleveland Cavaliers', 'W': '37', 'L': '45'}, {'index': 'East', 'teamname': 'Detroit Pistons', 'W': '16', 'L': '66'}, {'index': 'West', 'teamname': 'Milwaukee Bucks', 'W': '

In [5]:
import warnings
warnings.simplefilter('ignore')

# imports

import pandas as pd
import numpy as np

from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import Normalizer

In [6]:
#read csv files in
df = pd.read_csv("/kaggle/input/nbadata/NBA_data_all_19-20.csv")
df2020 = pd.read_csv("/kaggle/input/nbadata/NBA_data_all_80-19.csv")

In [7]:
# creating new columns
df["playoffs_y_n"] = df["Team"].map(lambda x: 1 if "*" in x else 0)
df["Team"] = df["Team"].str.replace("*", "")
df["W%"] = df["W"]/df["G"]

df2020["playoffs_y_n"] = df2020["Team"].map(lambda x: 1 if "*" in x else 0)
df2020["Team"] = df2020["Team"].str.replace("*", "")
df2020["W%"] = df2020["W"]/df2020["G"]

In [8]:
# scale stats by year
scaler = QuantileTransformer(output_distribution='normal')
normalizer = Normalizer()

norm_scaled_dfs = []

# These columns will not be scaled/normalized.
stay_columns = ["RK", "Team", "Year", "Conf", "playoffs_y_n", "G", "W", "L"]

a = df["Year"].min()
b = df["Year"].max()
x = b - a + 1
for i in range (x):
    year = a+i
    df_year = df.loc[df["Year"] == year]
    stay_df = df_year[stay_columns].reset_index(drop=True)   
    change_df = df_year.drop(columns=stay_columns).reset_index(drop=True)
    
#   scale data
    scaled_df = pd.DataFrame(scaler.fit_transform(change_df), columns=change_df.columns)
#   normalize data
    normalized_df = pd.DataFrame(normalizer.fit_transform(scaled_df), columns=scaled_df.columns)    
    
    merged_df = stay_df.merge(change_df, left_index=True, right_index=True)
    merged_df = merged_df.merge(normalized_df, left_index=True, right_index=True, suffixes = ('', '_scalnorm'))
    
    norm_scaled_dfs.append(merged_df)

final_df = pd.concat(norm_scaled_dfs, axis=0)
final_df = final_df.dropna()

# scale 2020 df
stay_df2020 = df2020[stay_columns].reset_index(drop=True)   
change_df2020 = df2020.drop(columns=stay_columns).reset_index(drop=True)

scaled_df2020 = pd.DataFrame(scaler.fit_transform(change_df2020), columns=change_df2020.columns)
normalized_df = pd.DataFrame(normalizer.fit_transform(scaled_df2020), columns=scaled_df2020.columns) 

merged_df2020 = stay_df2020.merge(change_df2020, left_index=True, right_index=True)
merged_df2020 = merged_df2020.merge(normalized_df, left_index=True, right_index=True, suffixes = ('', '_scalnorm'))
final_df2020 = merged_df2020 

In [9]:
final_df2020.to_csv("/kaggle/working/preprocessed_2020_Wperc.csv")
final_df.to_csv("/kaggle/working/preprocessed_80-19_Wperc.csv")

In [10]:
final_df.columns

Index(['RK', 'Team', 'Year', 'Conf', 'playoffs_y_n', 'G', 'W', 'L', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'W%',
       'MP_scalnorm', 'FG_scalnorm', 'FGA_scalnorm', 'FG%_scalnorm',
       '3P_scalnorm', '3PA_scalnorm', '3P%_scalnorm', '2P_scalnorm',
       '2PA_scalnorm', '2P%_scalnorm', 'FT_scalnorm', 'FTA_scalnorm',
       'FT%_scalnorm', 'ORB_scalnorm', 'DRB_scalnorm', 'AST_scalnorm',
       'STL_scalnorm', 'BLK_scalnorm', 'TOV_scalnorm', 'PF_scalnorm',
       'PTS_scalnorm', 'W%_scalnorm'],
      dtype='object')

In [11]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import joblib

In [12]:
model_df = pd.read_csv("/kaggle/working/preprocessed_2020_Wperc.csv", index_col = 0)
test_df = pd.read_csv("/kaggle/working/preprocessed_80-19_Wperc.csv", index_col = 0)

In [13]:
model_df.head()
test_df.head()
model_df.columns

Index(['RK', 'Team', 'Year', 'Conf', 'playoffs_y_n', 'G', 'W', 'L', 'MP', 'FG',
       'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'W%',
       'MP_scalnorm', 'FG_scalnorm', 'FGA_scalnorm', 'FG%_scalnorm',
       '3P_scalnorm', '3PA_scalnorm', '3P%_scalnorm', '2P_scalnorm',
       '2PA_scalnorm', '2P%_scalnorm', 'FT_scalnorm', 'FTA_scalnorm',
       'FT%_scalnorm', 'ORB_scalnorm', 'DRB_scalnorm', 'AST_scalnorm',
       'STL_scalnorm', 'BLK_scalnorm', 'TOV_scalnorm', 'PF_scalnorm',
       'PTS_scalnorm', 'W%_scalnorm'],
      dtype='object')

In [14]:
drop_columns = ['playoffs_y_n', 'RK', 'Team', 'Year', 'Conf',
                'G', 'W', 'L', 
                'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 
                'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                'W%',
               ]
df_east = model_df.loc[model_df["Conf"] == "East"]

train_M3E = df_east.loc[df_east["Year"] < 2019]
test_M3E = df_east.loc[df_east["Year"] >= 2019]

X_train_M3E = train_M3E.drop(columns=drop_columns)
X_test_M3E = test_M3E.drop(columns=drop_columns)

y_train_M3E = train_M3E[['playoffs_y_n']]
y_test_M3E = test_M3E[['playoffs_y_n']]

X_train_M3E.columns

Index(['MP_scalnorm', 'FG_scalnorm', 'FGA_scalnorm', 'FG%_scalnorm',
       '3P_scalnorm', '3PA_scalnorm', '3P%_scalnorm', '2P_scalnorm',
       '2PA_scalnorm', '2P%_scalnorm', 'FT_scalnorm', 'FTA_scalnorm',
       'FT%_scalnorm', 'ORB_scalnorm', 'DRB_scalnorm', 'AST_scalnorm',
       'STL_scalnorm', 'BLK_scalnorm', 'TOV_scalnorm', 'PF_scalnorm',
       'PTS_scalnorm', 'W%_scalnorm'],
      dtype='object')

In [15]:
print ("Logistic - East")
log_m3East = LogisticRegression()
log_m3East.fit(X_train_M3E, y_train_M3E)

log_east_train_score = log_m3East.score(X_train_M3E, y_train_M3E)
log_east_test_score = log_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {log_east_train_score}")
print(f"Testing Data Score: {log_east_test_score}")

print ("RFC - East")
rfc_m3East= RandomForestClassifier()
rfc_m3East.fit(X_train_M3E, y_train_M3E)

rfc_east_train_score = rfc_m3East.score(X_train_M3E, y_train_M3E)
rfc_east_test_score = rfc_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {rfc_east_train_score}")
print(f"Testing Data Score: {rfc_east_test_score}")

print ("SVC - East")
svc_m3East = SVC(kernel='linear', probability=True)
svc_m3East.fit(X_train_M3E, y_train_M3E)

svc_east_train_score = svc_m3East.score(X_train_M3E, y_train_M3E)
svc_east_test_score = svc_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {svc_east_train_score}")
print(f"Testing Data Score: {svc_east_test_score}")

east_scores = [["Train", log_east_train_score, rfc_east_train_score, svc_east_train_score],
               ["Test", log_east_test_score, rfc_east_test_score, svc_east_test_score]]
df_east_scores = pd.DataFrame(east_scores, columns = ['Type','Logistic', 'RFC', 'SVM']) 
df_east_scores

Logistic - East
Training Data Score: 0.9107806691449815
Testing Data Score: 0.8
RFC - East
Training Data Score: 1.0
Testing Data Score: 0.9333333333333333
SVC - East
Training Data Score: 0.9089219330855018
Testing Data Score: 0.8


,Type,Logistic,RFC,SVM
0,Train,0.910781,1.000000,0.908922
1,Test,0.800000,0.933333,0.800000


In [16]:
from sklearn.naive_bayes import GaussianNB

df_east = model_df.loc[model_df["Conf"] == "East"]

train_M3E = df_east.loc[df_east["Year"] < 2019]
test_M3E = df_east.loc[df_east["Year"] >= 2019]

X_train_M3E = train_M3E.drop(columns=drop_columns)
X_test_M3E = test_M3E.drop(columns=drop_columns)

y_train_M3E = train_M3E[['playoffs_y_n']]
y_test_M3E = test_M3E[['playoffs_y_n']]

X_train_M3E.columns

print ("Logistic - East")
log_m3East = LogisticRegression()
log_m3East.fit(X_train_M3E, y_train_M3E)

log_east_train_score = log_m3East.score(X_train_M3E, y_train_M3E)
log_east_test_score = log_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {log_east_train_score}")
print(f"Testing Data Score: {log_east_test_score}")

print ("RFC - East")
rfc_m3East= RandomForestClassifier()
rfc_m3East.fit(X_train_M3E, y_train_M3E)

rfc_east_train_score = rfc_m3East.score(X_train_M3E, y_train_M3E)
rfc_east_test_score = rfc_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {rfc_east_train_score}")
print(f"Testing Data Score: {rfc_east_test_score}")

print ("SVC - East")
svc_m3East = SVC(kernel='linear', probability=True)
svc_m3East.fit(X_train_M3E, y_train_M3E)

svc_east_train_score = svc_m3East.score(X_train_M3E, y_train_M3E)
svc_east_test_score = svc_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {svc_east_train_score}")
print(f"Testing Data Score: {svc_east_test_score}")

print("Naive Bayes - East")
nb_m3East = GaussianNB()
nb_m3East.fit(X_train_M3E, y_train_M3E)

nb_east_train_score = nb_m3East.score(X_train_M3E, y_train_M3E)
nb_east_test_score = nb_m3East.score(X_test_M3E, y_test_M3E)

print(f"Training Data Score: {nb_east_train_score}")
print(f"Testing Data Score: {nb_east_test_score}")

east_scores = [["Train", log_east_train_score, rfc_east_train_score, svc_east_train_score, nb_east_train_score],
               ["Test", log_east_test_score, rfc_east_test_score, svc_east_test_score, nb_east_test_score]]
df_east_scores = pd.DataFrame(east_scores, columns=['Type', 'Logistic', 'RFC', 'SVM', 'Naive Bayes'])
df_east_scores


Logistic - East
Training Data Score: 0.9107806691449815
Testing Data Score: 0.8
RFC - East
Training Data Score: 1.0
Testing Data Score: 0.9333333333333333
SVC - East
Training Data Score: 0.9089219330855018
Testing Data Score: 0.8
Naive Bayes - East
Training Data Score: 0.845724907063197
Testing Data Score: 0.7333333333333333


,Type,Logistic,RFC,SVM,Naive Bayes
0,Train,0.910781,1.000000,0.908922,0.845725
1,Test,0.800000,0.933333,0.800000,0.733333


In [17]:
feature_names_M3E = X_train_M3E.columns

print ("Log - East")
coeff_log = log_m3East.coef_
importance_log = coeff_log[0]
abs_importance_log = abs(importance_log)
importance_list_log = list(zip(feature_names_M3E,importance_log, abs_importance_log))
importance_list_log.sort(key=lambda x:x[2], reverse = True)
for i in range (5):
    print (importance_list_log[i])

print ("RFC - East")
rfc_importances = rfc_m3East.feature_importances_
rfc_importances = sorted(zip(rfc_importances, feature_names_M3E), reverse=True)
for i in range (5):
    print (rfc_importances[i])
    
print ("Naive Bayes - East")
nb_importances = nb_m3East.theta_[0] - nb_m3East.theta_[1]
abs_importance_nb = abs(nb_importances)
importance_list_nb = list(zip(feature_names_M3E, nb_importances, abs_importance_nb))
importance_list_nb.sort(key=lambda x:x[2], reverse = True)
for i in range(5):
    print(importance_list_nb[i])


print ("SVM - East")
coeff_svm = svc_m3East.coef_
importance_svm = coeff_svm[0]
abs_importance_svm = abs(importance_svm)
importance_list_svm = list(zip(feature_names_M3E,importance_svm, abs_importance_svm))
importance_list_svm.sort(key=lambda x:x[2], reverse = True)
for i in range (5):
    print (importance_list_svm[i])

Log - East
('W%_scalnorm', 8.104748931917317, 8.104748931917317)
('TOV_scalnorm', -1.372746005262575, 1.372746005262575)
('DRB_scalnorm', 1.238742580332647, 1.238742580332647)
('STL_scalnorm', 1.238549987116313, 1.238549987116313)
('2P%_scalnorm', 1.0726995983410785, 1.0726995983410785)
RFC - East
(0.48833745167336373, 'W%_scalnorm')
(0.04541380286903718, '2P%_scalnorm')
(0.04259926730456973, 'FG%_scalnorm')
(0.04208554411938319, 'DRB_scalnorm')
(0.03271336993230447, 'FT_scalnorm')
Naive Bayes - East
('W%_scalnorm', -0.32475687510708273, 0.32475687510708273)
('2P%_scalnorm', -0.14560106243165127, 0.14560106243165127)
('FG%_scalnorm', -0.13741939589148225, 0.13741939589148225)
('DRB_scalnorm', -0.09560407299716327, 0.09560407299716327)
('AST_scalnorm', -0.09411686374693087, 0.09411686374693087)
SVM - East
('W%_scalnorm', 8.012245034170471, 8.012245034170471)
('TOV_scalnorm', -1.0870566295854243, 1.0870566295854243)
('ORB_scalnorm', 1.0012041321510445, 1.0012041321510445)
('STL_scalnorm'

In [18]:
east_predictions_2019 = test_M3E[["Team", "Year", "playoffs_y_n"]]

log_probability = log_m3East.predict_proba(X_test_M3E)[:,1].tolist()
rfc_probability = rfc_m3East.predict_proba(X_test_M3E)[:,1].tolist()
svm_probability = svc_m3East.predict_proba(X_test_M3E)[:,1].tolist()
nb_probability = nb_m3East.predict_proba(X_test_M3E)[:,1].tolist()

log_prediction = log_m3East.predict(X_test_M3E).tolist()
rfc_prediction = rfc_m3East.predict(X_test_M3E).tolist()
svm_prediction = svc_m3East.predict(X_test_M3E).tolist()
nb_prediction = nb_m3East.predict(X_test_M3E).tolist()

east_predictions_2019["prediction_log"] = log_prediction
east_predictions_2019["prob_log"] = log_probability
east_predictions_2019["prediction_rfc"] = rfc_prediction
east_predictions_2019["prob_rfc"] = rfc_probability
east_predictions_2019["prediction_svm"] = svm_prediction
east_predictions_2019["prob_svm"] = svm_probability
east_predictions_2019["prediction_nb"] = nb_prediction
east_predictions_2019["prob_nb"] = nb_probability

east_predictions_2019.loc[east_predictions_2019["playoffs_y_n"] == 1]
east_predictions_2019.sort_values("prob_log", ascending = False)
east_predictions_2019.sort_values("prob_rfc", ascending = False)
east_predictions_2019.sort_values("prob_svm", ascending = False)
east_predictions_2019.sort_values("prob_nb", ascending = False)


,Team,Year,playoffs_y_n,prediction_log,prob_log,prediction_rfc,prob_rfc,prediction_svm,prob_svm,prediction_nb,prob_nb
1081,Toronto Raptors,2019,1,1,0.971792,1,0.97,1,0.999999,1,0.999990
1074,Milwaukee Bucks,2019,1,1,0.962367,1,0.97,1,0.999991,1,0.999963
1077,Philadelphia 76ers,2019,1,1,0.952179,1,0.95,1,0.996343,1,0.999940
1087,Boston Celtics,2019,1,1,0.898730,1,0.95,1,0.985054,1,0.999458
1095,Indiana Pacers,2019,1,1,0.865617,1,0.90,1,0.960630,1,0.999056
1083,Washington Wizards,2019,0,1,0.633346,0,0.38,1,0.526629,1,0.995474
1092,Charlotte Hornets,2019,0,1,0.739483,1,0.56,1,0.794352,1,0.990822
1088,Brooklyn Nets,2019,1,1,0.721743,1,0.74,1,0.842636,1,0.987468
1097,Orlando Magic,2019,1,1,0.717698,1,0.78,1,0.795345,1,0.977152
1098,Detroit Pistons,2019,1,1,0.640222,1,0.68,1,0.729435,1,0.949591


In [19]:
test_df_east = test_df.loc[test_df["Conf"] == "East"]
east_predictions_2020 = test_df_east[["Team", "Year", "playoffs_y_n"]]

X_test_M3E_2020 = test_df_east.drop(columns=drop_columns)

log_probability = log_m3East.predict_proba(X_test_M3E_2020)[:,1].tolist()
rfc_probability = rfc_m3East.predict_proba(X_test_M3E_2020)[:,1].tolist()
svm_probability = svc_m3East.predict_proba(X_test_M3E_2020)[:,1].tolist()
nb_probability = nb_m3East.predict_proba(X_test_M3E_2020)[:,1].tolist()



log_prediction = log_m3East.predict(X_test_M3E_2020).tolist()
rfc_prediction = rfc_m3East.predict(X_test_M3E_2020).tolist()
svm_prediction = svc_m3East.predict(X_test_M3E_2020).tolist()
nb_prediction = nb_m3East.predict(X_test_M3E_2020).tolist()

east_predictions_2020["prediction_log"] = log_prediction
east_predictions_2020["prob_log"] = log_probability
east_predictions_2020["prediction_rfc"] = rfc_prediction
east_predictions_2020["prob_rfc"] = rfc_probability
east_predictions_2020["prediction_svm"] = svm_prediction
east_predictions_2020["prob_svm"] = svm_probability
east_predictions_2020["prediction_nb"] = nb_prediction
east_predictions_2020["prob_nb"] = nb_probability

east_predictions_2020.loc[east_predictions_2020["playoffs_y_n"] == 1]
east_predictions_2020.sort_values("prob_log", ascending = False)
east_predictions_2020.sort_values("prob_rfc", ascending = False)
east_predictions_2020.sort_values("prob_svm", ascending = False)
east_predictions_2020.sort_values("prob_nb", ascending = False)

,Team,Year,playoffs_y_n,prediction_log,prob_log,prediction_rfc,prob_rfc,prediction_svm,prob_svm,prediction_nb,prob_nb
1,Milwaukee Bucks,2020,1,1,0.992599,1,0.90,1,1.000000,1,1.000000
13,Toronto Raptors,2020,1,1,0.993891,1,0.96,1,1.000000,1,0.999998
12,Boston Celtics,2020,1,1,0.994211,1,0.86,1,1.000000,1,0.999991
10,Miami Heat,2020,0,1,0.920309,1,0.93,1,0.992263,1,0.997900
17,Philadelphia 76ers,2020,0,1,0.949090,1,0.92,1,0.999989,1,0.997274
18,Indiana Pacers,2020,0,1,0.612349,1,0.77,1,0.755768,1,0.926671
21,Brooklyn Nets,2020,0,1,0.609674,1,0.81,1,0.769487,0,0.412143
27,Orlando Magic,2020,0,1,0.660621,1,0.78,1,0.690571,0,0.325382
23,New York Knicks,2020,0,0,0.386537,0,0.19,0,0.377482,0,0.223404
7,Washington Wizards,2020,0,0,0.234377,0,0.41,0,0.058241,0,0.217459


In [20]:
drop_columns = ['playoffs_y_n', 'RK', 'Team', 'Year', 'Conf',
                'G', 'W', 'L', 
                'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 
                'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                'W%',
               ]

df_west = model_df.loc[model_df["Conf"] == "West"]

train_M3W = df_west.loc[df_west["Year"] < 2019]
test_M3W = df_west.loc[df_west["Year"] >= 2019]

X_train_M3W = train_M3W.drop(columns=drop_columns)
X_test_M3W = test_M3W.drop(columns=drop_columns)

y_train_M3W = train_M3W[['playoffs_y_n']]
y_test_M3W = test_M3W[['playoffs_y_n']]

X_train_M3W.columns

Index(['MP_scalnorm', 'FG_scalnorm', 'FGA_scalnorm', 'FG%_scalnorm',
       '3P_scalnorm', '3PA_scalnorm', '3P%_scalnorm', '2P_scalnorm',
       '2PA_scalnorm', '2P%_scalnorm', 'FT_scalnorm', 'FTA_scalnorm',
       'FT%_scalnorm', 'ORB_scalnorm', 'DRB_scalnorm', 'AST_scalnorm',
       'STL_scalnorm', 'BLK_scalnorm', 'TOV_scalnorm', 'PF_scalnorm',
       'PTS_scalnorm', 'W%_scalnorm'],
      dtype='object')

In [21]:
print("Logistic Regression - West")
log_m3West = LogisticRegression()
log_m3West.fit(X_train_M3W, y_train_M3W)

log_west_train_score = log_m3West.score(X_train_M3W, y_train_M3W)
log_west_test_score = log_m3West.score(X_test_M3W, y_test_M3W)

print(f"Training Data Score: {log_west_train_score}")
print(f"Testing Data Score: {log_west_test_score}")

print("Random Forest - West")
rfc_m3West= RandomForestClassifier()
rfc_m3West.fit(X_train_M3W, y_train_M3W)

rfc_west_train_score = rfc_m3West.score(X_train_M3W, y_train_M3W)
rfc_west_test_score = rfc_m3West.score(X_test_M3W, y_test_M3W)

print(f"Training Data Score: {rfc_west_train_score}")
print(f"Testing Data Score: {rfc_west_test_score}")

print("SVM - West")
svc_m3West = SVC(kernel='linear', probability=True)
svc_m3West.fit(X_train_M3W, y_train_M3W)

svc_west_train_score = svc_m3West.score(X_train_M3W, y_train_M3W)
svc_west_test_score = svc_m3West.score(X_test_M3W, y_test_M3W)

print(f"Training Data Score: {svc_west_train_score}")
print(f"Testing Data Score: {svc_west_test_score}")

print("Naive Bayes - West")
nb_m3West = GaussianNB()
nb_m3West.fit(X_train_M3W, y_train_M3W)

nb_west_train_score = nb_m3West.score(X_train_M3W, y_train_M3W)
nb_west_test_score = nb_m3West.score(X_test_M3W, y_test_M3W)

print(f"Training Data Score: {nb_west_train_score}")
print(f"Testing Data Score: {nb_west_test_score}")

west_scores = [["Train", log_west_train_score, rfc_west_train_score, svc_west_train_score, nb_west_train_score],
               ["Test", log_west_test_score, rfc_west_test_score, svc_west_test_score, nb_west_test_score]]
df_west_scores = pd.DataFrame(west_scores, columns = ['Type', 'Logistic', 'RFC', 'SVM', 'Naive Bayes']) 
df_west_scores


Logistic Regression - West
Training Data Score: 0.9328358208955224
Testing Data Score: 0.6666666666666666
Random Forest - West
Training Data Score: 1.0
Testing Data Score: 1.0
SVM - West
Training Data Score: 0.9402985074626866
Testing Data Score: 0.6666666666666666
Naive Bayes - West
Training Data Score: 0.8656716417910447
Testing Data Score: 0.6


,Type,Logistic,RFC,SVM,Naive Bayes
0,Train,0.932836,1.0,0.940299,0.865672
1,Test,0.666667,1.0,0.666667,0.600000


In [22]:
feature_names_M3W = X_train_M3W.columns

print ("Log - West")
coeff_log = log_m3West.coef_
importance_log = coeff_log[0]
abs_importance_log = abs(importance_log)
importance_list_log = list(zip(feature_names_M3W,importance_log, abs_importance_log))
importance_list_log.sort(key=lambda x:x[2], reverse = True)
for i in range (5):
    print (importance_list_log[i])

print ("RFC - West")
rfc_importances = rfc_m3West.feature_importances_
rfc_importances = sorted(zip(rfc_importances, feature_names_M3W), reverse=True)
for i in range (5):
    print (rfc_importances[i])

print ("SVM - West")
coeff_svm = svc_m3West.coef_
importance_svm = coeff_svm[0]
abs_importance_svm = abs(importance_svm)
importance_list_svm = list(zip(feature_names_M3W,importance_svm, abs_importance_svm))
importance_list_svm.sort(key=lambda x:x[2], reverse = True)
for i in range (5):
    print (importance_list_svm[i])
    
    
print ("Naive Bayes - East")
nb_importances = nb_m3West.theta_[0] - nb_m3West.theta_[1]
abs_importance_nb = abs(nb_importances)
importance_list_nb = list(zip(feature_names_M3E, nb_importances, abs_importance_nb))
importance_list_nb.sort(key=lambda x:x[2], reverse = True)
for i in range(5):
    print(importance_list_nb[i])


Log - West
('W%_scalnorm', 7.374487625940595, 7.374487625940595)
('STL_scalnorm', 2.088668833153001, 2.088668833153001)
('TOV_scalnorm', -1.990071855873756, 1.990071855873756)
('AST_scalnorm', 1.8159906990809547, 1.8159906990809547)
('FGA_scalnorm', -1.121263798674797, 1.121263798674797)
RFC - West
(0.42622798951081897, 'W%_scalnorm')
(0.08232078390522629, '2P%_scalnorm')
(0.0641965994199846, 'FG%_scalnorm')
(0.04610576955756737, 'AST_scalnorm')
(0.04088446060035993, 'STL_scalnorm')
SVM - West
('W%_scalnorm', 6.951141866415467, 6.951141866415467)
('TOV_scalnorm', -1.8285516372501993, 1.8285516372501993)
('AST_scalnorm', 1.7045572423370454, 1.7045572423370454)
('STL_scalnorm', 1.5195206740606266, 1.5195206740606266)
('FGA_scalnorm', -0.9923963654536521, 0.9923963654536521)
Naive Bayes - East
('W%_scalnorm', -0.3775489767804484, 0.3775489767804484)
('2P%_scalnorm', -0.19240941346124396, 0.19240941346124396)
('FG%_scalnorm', -0.1803357068990067, 0.1803357068990067)
('AST_scalnorm', -0.161

In [23]:
west_predictions_2019 = test_M3W[["Team", "Year", "playoffs_y_n"]]

log_probability = log_m3West.predict_proba(X_test_M3W)[:,1].tolist()
rfc_probability = rfc_m3West.predict_proba(X_test_M3W)[:,1].tolist()
svm_probability = svc_m3West.predict_proba(X_test_M3W)[:,1].tolist()
nb_probability = nb_m3West.predict_proba(X_test_M3E)[:,1].tolist()

log_prediction = log_m3West.predict(X_test_M3W).tolist()
rfc_prediction = rfc_m3West.predict(X_test_M3W).tolist()
svm_prediction = svc_m3West.predict(X_test_M3W).tolist()
nb_prediction = nb_m3West.predict(X_test_M3E).tolist()

west_predictions_2019["prediction_log"] = log_prediction
west_predictions_2019["prob_log"] = log_probability
west_predictions_2019["prediction_rfc"] = rfc_prediction
west_predictions_2019["prob_rfc"] = rfc_probability
west_predictions_2019["prediction_svm"] = svm_prediction
west_predictions_2019["prob_svm"] = svm_probability
west_predictions_2019["prediction_nb"] = nb_prediction
west_predictions_2019["prob_nb"] = nb_probability

west_predictions_2019.loc[west_predictions_2019["playoffs_y_n"] == 1]
west_predictions_2019.sort_values("prob_log", ascending = False)
west_predictions_2019.sort_values("prob_rfc", ascending = False)
west_predictions_2019.sort_values("prob_svm", ascending = False)
west_predictions_2019.sort_values("prob_nb", ascending = False)

,Team,Year,playoffs_y_n,prediction_log,prob_log,prediction_rfc,prob_rfc,prediction_svm,prob_svm,prediction_nb,prob_nb
1078,Los Angeles Clippers,2019,1,1,0.789640,1,0.76,1,0.936872,1,0.999927
1075,Golden State Warriors,2019,1,1,0.919853,1,0.88,1,0.994675,1,0.999741
1076,New Orleans Pelicans,2019,0,1,0.516932,0,0.25,1,0.465119,1,0.999718
1082,Sacramento Kings,2019,0,1,0.562047,0,0.28,1,0.527346,1,0.998413
1089,Los Angeles Lakers,2019,0,1,0.513535,0,0.30,1,0.480896,1,0.998392
1079,Portland Trail Blazers,2019,1,1,0.832026,1,0.79,1,0.972873,1,0.972253
1086,Minnesota Timberwolves,2019,0,1,0.625418,0,0.31,1,0.729952,1,0.938594
1090,Utah Jazz,2019,1,1,0.890191,1,0.77,1,0.990595,1,0.920578
1084,Houston Rockets,2019,1,1,0.755274,1,0.77,1,0.958981,1,0.872582
1080,Oklahoma City Thunder,2019,1,1,0.838145,1,0.72,1,0.977023,1,0.680076


In [24]:
test_df_west = test_df.loc[test_df["Conf"] == "West"]
west_predictions_2020 = test_df_west[["Team", "Year", "playoffs_y_n"]]

X_test_M3W_2020 = test_df_west.drop(columns=drop_columns)

log_probability = log_m3West.predict_proba(X_test_M3W_2020)[:,1].tolist()
rfc_probability = rfc_m3West.predict_proba(X_test_M3W_2020)[:,1].tolist()
svm_probability = svc_m3West.predict_proba(X_test_M3W_2020)[:,1].tolist()
nb_probability = nb_m3West.predict_proba(X_test_M3E)[:,1].tolist()

log_prediction = log_m3West.predict(X_test_M3W_2020).tolist()
rfc_prediction = rfc_m3West.predict(X_test_M3W_2020).tolist()
svm_prediction = svc_m3West.predict(X_test_M3W_2020).tolist()
nb_prediction = nb_m3West.predict(X_test_M3E).tolist()

west_predictions_2020["prediction_log"] = log_prediction
west_predictions_2020["prob_log"] = log_probability
west_predictions_2020["prediction_rfc"] = rfc_prediction
west_predictions_2020["prob_rfc"] = rfc_probability
west_predictions_2020["prediction_svm"] = svm_prediction
west_predictions_2020["prob_svm"] = svm_probability
west_predictions_2020["prediction_nb"] = nb_prediction
west_predictions_2020["prob_nb"] = nb_probability

west_predictions_2020.loc[west_predictions_2020["playoffs_y_n"] == 1]
west_predictions_2020.sort_values("prob_log", ascending = False)
west_predictions_2020.sort_values("prob_rfc", ascending = False)
west_predictions_2020.sort_values("prob_svm", ascending = False)
west_predictions_2020.sort_values("prob_nb", ascending = False)
west_predictions_2020["avg_prob"] = (west_predictions_2020["prob_log"] +
                                     west_predictions_2020["prob_rfc"] +
                                     west_predictions_2020["prob_svm"] +
                                     west_predictions_2020["prob_nb"]) / 4

west_predictions_2020.sort_values("avg_prob", ascending=False)


,Team,Year,playoffs_y_n,prediction_log,prob_log,prediction_rfc,prob_rfc,prediction_svm,prob_svm,prediction_nb,prob_nb,avg_prob
14,Los Angeles Lakers,2020,1,1,0.971271,1,0.90,1,0.999997,1,0.998392,0.967415
15,Denver Nuggets,2020,0,1,0.964603,1,0.94,1,0.999995,1,0.920578,0.956294
0,Dallas Mavericks,2020,0,1,0.761881,1,0.78,1,0.930467,1,0.999741,0.868022
6,Los Angeles Clippers,2020,0,1,0.890361,1,0.89,1,0.988448,1,0.680076,0.862221
2,Houston Rockets,2020,0,1,0.651883,1,0.83,1,0.854895,1,0.999718,0.834124
8,Memphis Grizzlies,2020,0,1,0.658218,1,0.80,1,0.697502,1,0.998413,0.788533
9,Phoenix Suns,2020,0,1,0.732498,0,0.47,1,0.898838,1,0.872582,0.743480
20,Oklahoma City Thunder,2020,0,1,0.792492,1,0.83,1,0.930774,0,0.009752,0.640755
16,San Antonio Spurs,2020,0,1,0.808684,0,0.29,1,0.933328,0,0.420716,0.613182
19,Utah Jazz,2020,0,0,0.469932,1,0.79,0,0.246203,1,0.675178,0.545328
